In [3]:
from scipy.io import netcdf

import iris
import iris.coord_categorisation

import os
import logging

import numpy as np

import pickle
from pathlib import Path
logger = logging.getLogger(__name__)

In [4]:
def get_scenarios():
    """
        Return a List containing SCENARIO tag names
    """
    MODELS = [ 'clm', 'jules', 'orchidee' ]
    LU_SCENARIOS = [ 'nattobiobio','nattobioaff','nattobio','nattoaff','agtonat','agtobio','agtoaff' ]
    CLIM_SCENARIOS = [ 'ssp126', 'ssp370' ]
    CLIM_MODELS = [ 'mpi-esm1-2-hr','ukesm1-0-ll','ipsl-cm6a-lr' ]
    return CLIM_SCENARIOS, CLIM_MODELS

In [43]:
def main():

    LSM = 'clm'
    var='cSoil'

    if LSM=='jules':
        SCENARIO_TITLES = ['nolucbio','nolucaff','agtobio', 'agtonat', 'agtoaff', 'nattobio', 'nattoaff']
    elif LSM=='clm' or LSM=='orchidee':
        SCENARIO_TITLES = ['noluc','agtobio', 'agtonat', 'agtoaff', 'nattobio', 'nattoaff']
    elif LSM=='jsbach':
        SCENARIO_TITLES = ['noluc','agtobio', 'agtoaff', 'nattobio']

    nscenarios=len(SCENARIO_TITLES)
    ssp,esm=get_scenarios()


    if LSM=='jules':
        base_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+'nolucbio.pkl')
        base_file2= Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+'nolucaff.pkl')
        with open(base_file, 'rb') as f4:
            NOLUCBIO_DICT = pickle.load(f4)
        with open(base_file2, 'rb') as f5:
            NOLUCAFF_DICT = pickle.load(f5)
    else:
        base_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+'noluc.pkl')
        with open(base_file, 'rb') as f4:
            NOLUC_DICT = pickle.load(f4)
            
    for i0 in range(nscenarios):

        print(SCENARIO_TITLES[i0])
        my_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+SCENARIO_TITLES[i0]+'.pkl')
        tropics_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+SCENARIO_TITLES[i0]+'_tropics.pkl')
        temperate_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+SCENARIO_TITLES[i0]+'_temp.pkl')

    
        if temperate_file.is_file():
            # file exists, we can skip a bunch of steps:
            with open(my_file, 'rb') as f1:
                ENSEMBLE_MEAN = pickle.load(f1)
            with open(tropics_file, 'rb') as f2:
                TROPICS_MEAN = pickle.load(f2)
            with open(temperate_file, 'rb') as f3:    
                TEMPERATE_MEAN = pickle.load(f3)

        for i1 in range(0,2):
            temparray=np.array(list(ENSEMBLE_MEAN[ssp[i1]].values()))
            if LSM=='jules':
                if SCENARIO_TITLES[i0]=='nolucbio' or SCENARIO_TITLES[i0]=='agtobio' or SCENARIO_TITLES[i0]=='nattobio':
                    nolucarray=np.array(list(NOLUCBIO_DICT[ssp[i1]].values()))
                else:
                    nolucarray=np.array(list(NOLUCAFF_DICT[ssp[i1]].values()))
            else:
                nolucarray=np.array(list(NOLUC_DICT[ssp[i1]].values()))
            
#            delta_gb=(temparray[:,0,-1]-temparray[:,0,0]) - (nolucarray[:,0,-1]-nolucarray[:,0,0]) 
            delta_gb=(temparray[:,0,-1]-temparray[:,0,0])
           
        
            ssp_mean = np.mean(delta_gb)
            ssp_min = np.min(delta_gb)
            ssp_max = np.max(delta_gb)
        
            print('Global average,min,max change for:',ssp[i1],ssp_mean,ssp_min,ssp_max)
        print(' ')
        

In [44]:
if __name__ == "__main__":
    main()

noluc
Global average,min,max change for: ssp126 28.65168795838493 19.70645046718755 39.95606527546124
Global average,min,max change for: ssp370 29.522511130347993 12.435910808047083 52.731871927059274
 
agtobio
Global average,min,max change for: ssp126 21.634865179640656 12.907392893940596 32.832446342599724
Global average,min,max change for: ssp370 22.361725964345016 5.428477276227568 45.48222638125435
 
agtonat
Global average,min,max change for: ssp126 31.339161313713095 22.58635090847565 42.50255669807075
Global average,min,max change for: ssp370 33.16353206558082 16.0812145379532 56.28953058486377
 
agtoaff
Global average,min,max change for: ssp126 30.830915446770632 21.899356652300412 42.13880116487917
Global average,min,max change for: ssp370 33.420585085815524 16.271002273226713 56.79675168012045
 
nattobio
Global average,min,max change for: ssp126 10.521409772523233 0.9802668106412966 22.13293397328971
Global average,min,max change for: ssp370 8.788868148559686 -8.8684467086691

# 